# Game.py

In [1]:
#game.py
# Going to change to a class instead for ease?
import numpy as np
class dots_and_boxes():
  def __init__(self, n):
    self.n = n
    """
    Initialize the 5 layers that represent a game state
    board_tensor: shape (n, n, 5) with 4 edges per box and an extra channel for player ownership
                  channels are in [top, bottom, left, right, player owner]
    boxes_playerA: list of boxes owned by player A in (x,y)
    boxes_playerB: list of boxes owned by player B
    recent_edge: most recent edge drawn
    """
    self.board_tensor = np.zeros((n, n, 5), dtype=int)
    self.boxes_playerA = []
    self.boxes_playerB = []
    self.recent_edge = None
    self.On_Offensive = False
    self.overlapping_edges = set()
    self.non_overlapping_edges = set()
    self.initialize_edge_sets()


  def set_edge(self, r, c, edge_type):
    """
    Set the edge of box (r,c)
    edge_type: "top", "bottom", "left", "right"
    Automatically updates neighboring box if shared
    Updates recent_edge
    """
    n = self.n
    self.recent_edge = [r,c,edge_type]

    if edge_type == "top":
        self.board_tensor[r, c, 0] = 1
        if r > 0:
            self.board_tensor[r-1, c, 1] = 1  # bottom of box above
    elif edge_type == "bottom":
        self.board_tensor[r, c, 1] = 1
        if r < n-1:
            self.board_tensor[r+1, c, 0] = 1  # top of box below
    elif edge_type == "left":
        self.board_tensor[r, c, 2] = 1
        if c > 0:
          self.board_tensor[r, c-1, 3] = 1  # right of box to the left
    elif edge_type == "right":
        self.board_tensor[r, c, 3] = 1
        if c < n-1:
          self.board_tensor[r, c+1, 2] = 1  # left of box to the right

  def check_edge(self, r, c, edge_type):
    """
    Check if edge of box (r,c) is drawn
    edge_type: "top", "bottom", "left", "right"
    Returns true if available, false if not
    """
    edge_index = {"top": 0, "bottom": 1, "left": 2, "right": 3}
    if edge_type not in edge_index:
      return False
    return self.board_tensor[r, c, edge_index[edge_type]] == 0

  def check_box(self, r, c, edge_type, player):
    """
    Checks if all edges of box (r,c) are drawn
    If a box is created, assigns ownership
    Also checks if additonal box next to (r,c) is completed
    edge_type: "top", "bottom", "left", "right"
    player: 1,2
    Automatically updates boxes_playerA and boxes_playerB
    Automatically updates On_Offensive
    """
    n = self.n
    current_box = False

    if np.all(self.board_tensor[r, c, :4] == 1):
      current_box = True

      # Updating boxes_playerA or boxes_playerB
      self.board_tensor[r,c,4] = player # assigns ownership
      if player == 1:
        self.boxes_playerA.append((r,c))
      else:
        self.boxes_playerB.append((r,c))

    additional_box = False

    # Used to update the boxes under each player with the appropriate additional box
    additional_r = r
    additional_c = c

    if edge_type == "top":
      if r > 0:  # check box above
        if np.all(self.board_tensor[r-1, c, :4] == 1): # all edges are filled in
          additional_box = True
          self.board_tensor[r-1, c, 4] = player # assigns ownership
          additional_r = r-1

    elif edge_type == "bottom":
      if r < (n-1):  # check box below
        if np.all(self.board_tensor[r+1, c, :4] == 1): # all edges are filled in
          additional_box = True
          self.board_tensor [r+1, c, 4] = player # assigns ownership
          additional_r = r+1

    elif edge_type == "left":
      if c > 0: # check the box to the left
        if np.all(self.board_tensor[r,c-1, :4] == 1):
          additional_box = True
          self.board_tensor[r, c-1, 4] = player # assigns ownership
          additional_c = c-1

    elif edge_type == "right":
      if c < (n-1): # check the box to the right
        if np.all(self.board_tensor[r, c+1, :4] == 1):
          additional_box = True
          self.board_tensor [r, c+1, 4] = player # assigns ownership
          additional_c = c+1

    # Updating boxes_playerA or boxes_playerB
    if additional_box:
      if player == 1:
        self.boxes_playerA.append((additional_r,additional_c))
      else:
        self.boxes_playerB.append((additional_r,additional_c))

    # Updating On_Offensive
    self.On_Offensive = current_box or additional_box

    return int(current_box) + int(additional_box)


  def print_board_ascii(self, move, player, initial = False):
    """
    Print the current board
    move: move number
    player: player number
    initial: True if first time showing the board
    """
    n = self.n
    board_tensor = self.board_tensor
    BOX_WIDTH = 7

    if(initial):
      print()
      intro = "Welcome!"
      intro = intro.center(BOX_WIDTH * n + 5)
      print(intro)
      welcome = "Here is the board"
      welcome = welcome.center(BOX_WIDTH * n + 5)
      print(welcome)
    else:
      player_intro = f"Player {player} , Move {move}"
      player_intro = player_intro.center(BOX_WIDTH * n + 5)
      print(player_intro)
    for r in range(n):
        # Print top edges
        top_row = " "
        for c in range(n):
            top_row += "+" #prints all points
            top_row += "-" * BOX_WIDTH if board_tensor[r, c, 0] else " " * BOX_WIDTH # prints a horizontal line the same length as box width
        top_row += "+"
        print(top_row)

        # Print vertical edges and ownership/coordinates
        mid_row = " "
        for c in range(n):
            mid_row += "|" if board_tensor[r, c, 2] else " "

            # show coordinates unless box is owned (shows owner in that case)
            owner = board_tensor[r, c, 4]
            content = str(int(owner)) if owner else f"({r},{c})"
            content = content.center(BOX_WIDTH)
            mid_row += content

        # last right edge of the row
        mid_row += "|" if board_tensor[r, n-1, 3] else " "
        print(mid_row)

    # Print bottom edges of last row
    bottom_row = " "
    for c in range(n):
        bottom_row += "+"
        bottom_row += "-" * BOX_WIDTH if board_tensor[n-1, c, 1] else " " * BOX_WIDTH # prints a horizontal line the same length as box width
    bottom_row += "+"
    print(bottom_row)
    print()

  def player_turn(self, r, c, edge_type, player,  move):
  # Essentially just instantiates other functions that are used in a turn
    self.set_edge(r, c, edge_type)
    completed_box = self.check_box(r, c, edge_type, player) # if want to find out how many boxes made by a move
    self.print_board_ascii(move, player)
    return completed_box

  def initialize_edge_sets(self):
    """
    Initalizes both edge set types (overlapping and nonoverlapping)
    with the appropriate edge tuples based on n
    """
    n = self.n

    #initialize non-overlapping
    for r in range(n):
        for c in range(n):
            # top boundary
            if r == 0:
                self.non_overlapping_edges.add((r, c, "top"))
            # bottom boundary
            if r == n - 1:
                self.non_overlapping_edges.add((r, c, "bottom"))
            # left boundary
            if c == 0:
                self.non_overlapping_edges.add((r, c, "left"))
            # right boundary
            if c == n - 1:
                self.non_overlapping_edges.add((r, c, "right"))
    #initalize overlapping
    for r in range(n):
          for c in range(n):
              # top overlaps with box above
              if r > 0:
                  self.overlapping_edges.add((r, c, "top"))
              # bottom overlaps with box below
              if r < n - 1:
                  self.overlapping_edges.add((r, c, "bottom"))
              # left overlaps with box to the left
              if c > 0:
                  self.overlapping_edges.add((r, c, "left"))
              # right overlaps with box to the right
              if c < n - 1:
                  self.overlapping_edges.add((r, c, "right"))



In [2]:
# @title

# OLD CODE TESTING INITIALIZATION AND SETTING EDGE AND CHECKING IF BOXES COMPLETE AND DRAWING THEM
# Initialize 2x2 board
# board = init_box_tensor(2)

# completed_boxes_player1 = 0
# completed_boxes_player2 = 0

# # Draw the top edge of box (0,0)
# set_edge(board, 0, 0, "top")
# completed_boxes_player1 += check_box(board, 0, 0, "top", 1)
# print_board_ascii(board, 1, 1)

# # Draw the left edge of box (0,0)
# set_edge(board, 0, 0, "left")
# completed_boxes_player2 += check_box(board, 0, 0, "left", 2)
# print_board_ascii(board, 2, 2)

# # Draw the bottom edge of box (0,0)
# set_edge(board, 0, 0, "bottom")
# completed_boxes_player1 += check_box(board, 0, 0, "bottom", 1)
# print_board_ascii(board, 3, 1)

# # Draw the top edge of box (0,1)
# set_edge(board, 0, 1, "top")
# completed_boxes_player2 += check_box(board, 0, 1, "top", 2)
# print_board_ascii(board, 4, 2)

# # Draw the right edge of box (0,1)
# set_edge(board, 0, 1, "right")
# completed_boxes_player1 += check_box(board, 0, 1, "right", 1)
# print_board_ascii(board, 5, 1)

# # Draw the bottom edge of box (0,1)
# set_edge(board, 0, 1, "bottom")
# completed_boxes_player2 += check_box(board, 0, 1, "bottom", 2)
# print_board_ascii(board, 6, 2)

# # Draw the left edge of box (0,1) => completes 2 boxes for player 1
# set_edge(board, 0, 1, "left")
# completed_boxes_player1 += check_box(board, 0, 1, "left", 1)
# print_board_ascii(board, 7, 1)

# print(board[0,0])  # Should be [1,1,1,1] => all edges drawn
# print("player 1 has", completed_boxes_player1, "boxes") # Should be 2
# print("player 2 has", completed_boxes_player2, "boxes") # Should be 0


---
# Gameplay Loop

In [3]:
def gameplay():
# Initializing board and vars
  n = int(input("Enter board size to start: "))
  game = dots_and_boxes(n)
  move = 1
  player = 1
  game.print_board_ascii(move, player, initial = True)

# Players take turns until all boxes made
  while (len(game.boxes_playerA) + len(game.boxes_playerB)) < n**2:
    if(game.On_Offensive):
      print(f"Player {player}'s turn again!\n")
    else:
      print(f"Player {player}'s turn!")

    valid_move = False
    while not valid_move:
      try:
        # Getting parameters for edge to draw
        s = input("Enter the box to place edge as (row,column): ")
        s = s.strip()
        s = s.strip("(").strip(")")
        r,c = s.split(",")
        r = int(r)
        c = int(c)

        # Check if row coordinate is within bounds
        if r < 0 or r >= n:
          print(f"Invalid row coordinate! Please enter values between 0 and {n-1}.\n")
          continue

        # Check if column coordinate is within bounds
        if c < 0 or c >= n:
          print(f"Invalid column coordinate! Please enter values between 0 and {n-1}.\n")
          continue

        edge_type = input("Select what edge to place the line on from 'top', 'bottom', 'left', and 'right': ")
        print()

        # Check if edge_type is valid
        if edge_type not in ["top", "bottom", "left", "right"]:
          print("Invalid edge type! Please enter 'top', 'bottom', 'left', or 'right'.\n")
          continue

        # Check if edge is already drawn
        if not game.check_edge(r, c, edge_type):
          print("That edge is already drawn! Please choose a different edge.\n")
          continue

        # If all checks pass, the move is valid
        valid_move = True

      except ():
        continue

    # Making changes to board based on edge drawn
    _ = game.player_turn(r, c, edge_type, player, move)

    # Updating parameters for next turn
    move += 1
    if not game.On_Offensive:
      if player == 1:
        player = 2
      else:
        player = 1

  # Checking who won afer game ended
  if (len(game.boxes_playerA) > len(game.boxes_playerB)):
    print("Congratulations! Player 1 wins!")
  elif(len(game.boxes_playerB) > len(game.boxes_playerA)):
    print("Congratulations! Player 2 wins!")
  else:
    print("Woah! It's a tie!")

---
# Testing gameplay loop

In [4]:
#gameplay()

---
# Model.py


In [5]:
#model.py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# ----------------------------------------------------
# Residual block (AlphaZero style)
# ----------------------------------------------------
class ResidualBlock(nn.Module):

    def __init__(self, channels):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu  = nn.ReLU()

    def forward(self, x):

        residual = x

        x = self.conv1(x)
        x = self.relu(x)

        x = self.conv2(x)

        # Add skip connection
        x = x + residual

        x = self.relu(x)

        return x



# ----------------------------------------------------
# Main Network
# ----------------------------------------------------
class Connect2Model(nn.Module):

    def __init__(self, board_size, device):

        super(Connect2Model, self).__init__()

        self.device = device
        self.size = board_size                                    # n x n boxes x 5 channels
        n = board_size[0]

        self.action_size = 2 * n * (n + 1)

        # ---------------------------------------------
        # Initial convolution
        # ---------------------------------------------
        self.conv_input = nn.Conv2d(in_channels=6, out_channels=32,
                                    kernel_size=3, padding=1)
        self.relu = nn.ReLU()

        # ---------------------------------------------
        # Residual tower (3 blocks for now)
        # ---------------------------------------------
        self.res1 = ResidualBlock(32)
        self.res2 = ResidualBlock(32)
        self.res3 = ResidualBlock(32)

        # ---------------------------------------------
        # Policy head
        # ---------------------------------------------
        self.policy_conv = nn.Conv2d(32, 2, kernel_size=1)          # 1x1 conv
        self.policy_fc   = nn.Linear(2 * n * n, self.action_size)   # flatten → action vector

        # ---------------------------------------------
        # Value head
        # ---------------------------------------------
        self.value_conv = nn.Conv2d(32, 1, kernel_size=1)           # 1x1 conv
        self.value_fc1  = nn.Linear(1 * n * n, 64)
        self.value_fc2  = nn.Linear(64, 1)

        self.to(device)



    # ----------------------------------------------------
    # Forward pass
    # ----------------------------------------------------
    def forward(self, x):

        # Initial conv
        x = self.conv_input(x)
        x = self.relu(x)

        # Residual blocks
        x = self.res1(x)
        x = self.res2(x)
        x = self.res3(x)

        # --------------------------
        # POLICY HEAD
        # --------------------------
        p = self.policy_conv(x)               # shape: batch × 2 × n × n
        p = p.reshape(p.size(0), -1)             # flatten
        p = self.policy_fc(p)                 # output action logits

        # --------------------------
        # VALUE HEAD
        # --------------------------
        v = self.value_conv(x)                # batch × 1 × n × n
        v = v.view(v.size(0), -1)
        v = self.value_fc1(v)
        v = torch.tanh(self.value_fc2(v))     # scalar in [-1, 1]

        return p, v


# MCTS.py (network based - using policy,priors etc)


In [6]:
import numpy as np
import torch
import torch.nn.functional as F
import math
from copy import deepcopy

# ----------------------------------------------------
# Node in the MCTS
# ----------------------------------------------------
class MCTSNode:
    def __init__(self, state, parent=None, prior_prob=0):
        self.state = state  # game state
        self.parent = parent
        self.children = {}
        self.visit_count = 0
        self.value_sum = 0
        self.prior_prob = prior_prob

    def is_expanded(self):
        return len(self.children) > 0

    def value(self):
        if self.visit_count == 0:
            return 0
        return self.value_sum / self.visit_count

    def select_child(self, c_puct=1.4):
        """Select child with highest UCB score"""
        best_score = -float('inf')
        best_action = None
        best_child = None

        for action, child in self.children.items():
            # UCB formula: Q(s,a) + c_puct * P(s,a) * sqrt(N(s)) / (1 + N(s,a))
            q_value = child.value()
            u_value = c_puct * child.prior_prob * math.sqrt(self.visit_count) / (1 + child.visit_count)
            score = q_value + u_value

            if score > best_score:
                best_score = score
                best_action = action
                best_child = child

        return best_action, best_child

# ----------------------------------------------------
# MCTS Algorithm (using policy network to evaluate)
# ----------------------------------------------------
class MCTS:
    def __init__(self, model, device, num_simulations=100, c_puct=1.4):
        self.model = model
        self.num_simulations = num_simulations
        self.c_puct = c_puct
        self.device = device

    # ----------------------------------------------------
    # Helper functions to be used in the tree functions
    # ----------------------------------------------------

    """Function to convert the board representation to a number"""
    def decode_action(self, action, n):
        # Horizontal edges: 0 to n*(n+1)-1
        # These represent horizontal lines in the grid
        if action < n * (n + 1):
            row_of_edge = action // n
            col_of_edge = action % n

            # This horizontal edge is the top of box (row_of_edge, col_of_edge)
            # or the bottom of box (row_of_edge-1, col_of_edge)
            # We'll use it as top edge when possible
            if row_of_edge < n:
                return row_of_edge, col_of_edge, "top"
            else:
                return row_of_edge - 1, col_of_edge, "bottom"

        # Vertical edges: n*(n+1) to 2*n*(n+1)-1
        # These represent vertical lines in the grid
        else:
            action -= n * (n + 1)
            row_of_edge = action // (n + 1)
            col_of_edge = action % (n + 1)

            # This vertical edge is the left of box (row_of_edge, col_of_edge)
            # or the right of box (row_of_edge, col_of_edge-1)
            # We'll use it as left edge when possible
            if col_of_edge < n:
                return row_of_edge, col_of_edge, "left"
            else:
                # This is the rightmost edge
                return row_of_edge, col_of_edge - 1, "right"

    """Function to convert game state to neural network format"""
    def state_to_input(self, state, player):
        # Transpose to format that neural network expects: (5,n,n)
        board = np.transpose(state.board_tensor, (2, 0, 1)).astype(np.float32)

        # Add player perspective channels
        n = state.n
        player_channel = np.full((1,n,n), 1.0 if player == 1 else -1.0, dtype=np.float32)

        # Swap ownership channel based on player perspective
        ownership = board[4:5].copy()
        if player == 2:
            # From P2's perspective, swap the ownership encoding
            ownership = np.where(ownership == 1, -1, ownership)
            ownership = np.where(ownership == 2, 1, ownership)
            ownership = np.where(ownership == -1, 2, ownership)
            board = board.copy()
            board[4:5] = ownership

        board_with_player = np.concatenate([board, player_channel], axis=0)

        return board_with_player

    """Function to get binary mask of valid actions"""
    def get_valid_actions(self, state):
        n = state.n
        valid_actions = np.zeros(2 * n * (n + 1), dtype=np.float32)

        # Horizontal edges
        for r in range(n + 1):
            for c in range(n):
                action = r * n + c
                # Check if this is a valid edge to draw
                if r < n and state.check_edge(r, c, "top"):
                    valid_actions[action] = 1
                elif r > 0 and state.check_edge(r - 1, c, "bottom"):
                    valid_actions[action] = 1

        # Vertical edges
        for r in range(n):
            for c in range(n + 1):
                action = n * (n + 1) + r * (n + 1) + c
                # Check if this is a valid edge to draw
                if c < n and state.check_edge(r, c, "left"):
                    valid_actions[action] = 1
                elif c > 0 and state.check_edge(r, c - 1, "right"):
                    valid_actions[action] = 1

        return valid_actions

    """Function to check if game is over"""
    def is_terminal(self, state):
        total_boxes = state.n ** 2
        total_filled = len(state.boxes_playerA) + len(state.boxes_playerB)
        return total_filled == total_boxes

    """Function to get value for terminal state from perspective of player"""
    def get_terminal_value(self, state, player):
        if player == 1:
            score_diff = len(state.boxes_playerA) - len(state.boxes_playerB)
        else:
            score_diff = len(state.boxes_playerB) - len(state.boxes_playerA)

        if score_diff > 0:
            return 1.0
        elif score_diff < 0:
            return -1.0
        else:
            return 0.0

    # ----------------------------------------------------
    # Tree functions
    # ----------------------------------------------------

    """Function to use the policy network to evaluate the position"""
    def evaluate(self, state, player):
        self.model.eval()
        with torch.no_grad():
            # Prepare input
            board_input = self.state_to_input(state, player)
            board_tensor = torch.FloatTensor(board_input).unsqueeze(0).to(self.device)

            # Get policy and value from network
            policy_logits, value = self.model(board_tensor)
            policy_probs = F.softmax(policy_logits, dim=1).cpu().numpy()[0]
            value = value.item()

        return policy_probs, value

    """Function to backpropagate the value up the tree"""
    def backpropagate(self, search_path, value, player):
        for node in reversed(search_path):
            node.value_sum += value
            node.visit_count += 1
            value = -value  # Flip value for opponent

    """Function to run MCTS simulations and return action probabilities"""
    def search(self, root_state, player):
        root = MCTSNode(deepcopy(root_state))

        for _ in range(self.num_simulations):
            node = root
            search_path = [node]
            current_state = deepcopy(root_state)
            current_player = player

            # Selection: traverse tree using UCB
            while node.is_expanded():
                action, node = node.select_child(self.c_puct)
                search_path.append(node)

                # Apply action to state
                r, c, edge_type = self.decode_action(action, current_state.n)
                current_state.set_edge(r, c, edge_type)
                boxes_made = current_state.check_box(r, c, edge_type, current_player)

                # Switch player only if no box was completed
                if boxes_made == 0:
                    current_player = 3 - current_player # Switches 1 to 2 and 2 to 1

            # Check if game is over
            if self.is_terminal(current_state):
                value = self.get_terminal_value(current_state, player)
            else:
                # Expansion and Evaluation
                policy_probs, value = self.evaluate(current_state, current_player)
                valid_actions = self.get_valid_actions(current_state)

                # Mask invalid actions
                policy_probs = policy_probs * valid_actions
                if policy_probs.sum() > 0:
                    policy_probs = policy_probs / policy_probs.sum()

                # Expand node
                for action in range(len(policy_probs)):
                    if valid_actions[action] > 0:
                        node.children[action] = MCTSNode(
                            deepcopy(current_state),
                            parent=node,
                            prior_prob=policy_probs[action]
                        )

                # Adjust value based on current player perspective
                if current_player != player:
                    value = -value

            # Backpropagation
            self.backpropagate(search_path, value, player)

        # Return action probabilities based on visit counts
        return self.get_action_probs(root)

    """Function to get the acion probabilities from the visit counts"""
    def get_action_probs(self, root, temperature=1.0):
        action_size = 2 * root.state.n * (root.state.n + 1)
        visit_counts = np.zeros(action_size, dtype=np.float32)

        for action, child in root.children.items():
            visit_counts[action] = child.visit_count

        if temperature == 0:
            # Deterministic: pick most visited
            probs = np.zeros_like(visit_counts)
            probs[np.argmax(visit_counts)] = 1.0
            return probs
        else:
            # Apply temperature
            visit_counts = visit_counts ** (1.0 / temperature)
            if visit_counts.sum() > 0:
                probs = visit_counts / visit_counts.sum()
            else:
                probs = visit_counts
            return probs


---
# execute_episode for self.play() loop to generate training data

In [7]:
import copy
import random
import time

class SelfPlay:
    def __init__(self, game, model, device, num_simulations=100, temperature=1.0):
        self.game = game
        self.model = model
        self.device = device
        self.num_simulations = num_simulations
        self.temperature = temperature

    def execute_episode(self):
        """
        Play 1 full self-play game using MCTS, essentially 2 AI's versing eachother using MCTS.
        Returns a list of (state, move, z(final game outcome)) tuples.
        """
        training_examples = [] #to store raw moves/states during game
        state = self.game  # starting board
        cur_player = 1

        # mcts = MCTS(time_limit=1.0)
        mcts = MCTS(
            model=self.model,
            device=self.device,
            num_simulations=self.num_simulations
        )

        move_number = 1

        while True:

            canonical_state = copy.deepcopy(state.board_tensor)

            # Get move from MCTS
            # move = mcts.get_move(state, cur_player) # Pass cur_player to MCTS
            pi = mcts.search(state, cur_player)

            # Save example
            training_examples.append((canonical_state, pi, cur_player))

            # # Apply move
            # r, c, edge_type = move
            action = np.random.choice(len(pi), p=pi)
            r, c, edge_type = mcts.decode_action(action, state.n)

            print(f"\nMove {move_number} | Player {cur_player}")
            print(f"Chosen action index: {action}  |  Probability: {pi[action]:.3f}")
            print(f"Move: (r={r}, c={c}, edge='{edge_type}')\n")

            state.set_edge(r, c, edge_type)
            completed_boxes = state.check_box(r, c, edge_type, cur_player)

            # Print board after the move
            state.print_board_ascii(move_number, cur_player)
            move_number += 1

            # Determine next player based on completed box check
            if completed_boxes == 0:
                cur_player = 3 - cur_player  # switch player, if cur_player = 1, new cur_player=2, if 2, new cur_player=1
            # else, same player continues (extra turn)


            # # Check game-end condition
            # total_boxes = len(state.boxes_playerA) + len(state.boxes_playerB)
            # if total_boxes == state.n * state.n:
            #     result = 0
            #     if len(state.boxes_playerA) > len(state.boxes_playerB):
            #         result = 1
            #     elif len(state.boxes_playerB) > len(state.boxes_playerA):
            #         result = -1

            #     final_data = [] #training_examples, but final-outcome aware (who won??)
            #     for state, move, player in training_examples:
            #         z = result if player == 1 else -result
            #         final_data.append((state, move, z))
            #     return final_data

             # Check for terminal state
            if mcts.is_terminal(state):
                winner_val = mcts.get_terminal_value(state, 1)  # value for player 1

                final_data = []
                for state, pi, player in training_examples:
                    z = winner_val if player == 1 else -winner_val
                    final_data.append((state, pi, z))
                return final_data

#NOTE - This is just running 1 game, loop this over 1000 games eventually for full training

game = dots_and_boxes(2)
device = "cpu"
model = Connect2Model(board_size=(2, 2), device=device)

self_play = SelfPlay(game, model, device)
data = self_play.execute_episode()


Move 1 | Player 1
Chosen action index: 11  |  Probability: 0.101
Move: (r=1, c=1, edge='right')

 Player 1 , Move 1 
 +       +       +
   (0,0)   (0,1)  
 +       +       +
   (1,0)   (1,1) |
 +       +       +


Move 2 | Player 2
Chosen action index: 2  |  Probability: 0.111
Move: (r=1, c=0, edge='top')

 Player 2 , Move 2 
 +       +       +
   (0,0)   (0,1)  
 +-------+       +
   (1,0)   (1,1) |
 +       +       +


Move 3 | Player 1
Chosen action index: 9  |  Probability: 0.101
Move: (r=1, c=0, edge='left')

 Player 1 , Move 3 
 +       +       +
   (0,0)   (0,1)  
 +-------+       +
 | (1,0)   (1,1) |
 +       +       +


Move 4 | Player 2
Chosen action index: 10  |  Probability: 0.182
Move: (r=1, c=1, edge='left')

 Player 2 , Move 4 
 +       +       +
   (0,0)   (0,1)  
 +-------+       +
 | (1,0) | (1,1) |
 +       +       +


Move 5 | Player 1
Chosen action index: 8  |  Probability: 0.152
Move: (r=0, c=1, edge='right')

 Player 1 , Move 5 
 +       +       +
   (0,0)   (0,

---
# pure_MCTS.py (implemented from ai-service.py file in GitHub)

In [8]:
#pure_MCTS.py
import numpy as np
import time
import random
import math

class pureMCTSNode:
    """Node in the MCTS tree for dots_and_boxes game"""

    def __init__(self, game_state, current_player, parent=None, move=None):
        """
        game_state: dots_and_boxes object
        current_player: The player whose turn it is to make a move from this state (1 or 2)
        move: tuple (r, c, edge_type) that led to this node
        """
        self.game_state = game_state
        self.current_player = current_player
        self.parent = parent
        self.move = move

        self.children = []
        self.untried_moves = self.get_legal_moves()

        self.visits = 0
        self.wins = 0.0

    def get_legal_moves(self):
        """Get all legal moves from current state"""
        moves = []
        n = self.game_state.n

        for r in range(n):
            for c in range(n):
                for edge_type in ["top", "bottom", "left", "right"]:
                    if self.game_state.check_edge(r, c, edge_type):
                        moves.append((r, c, edge_type))

        return moves

    def is_fully_expanded(self):
        """Check if all children have been created"""
        return len(self.untried_moves) == 0

    def is_terminal(self):
        """Check if game is over"""
        n = self.game_state.n
        total_boxes = n ** 2
        total_filled = len(self.game_state.boxes_playerA) + len(self.game_state.boxes_playerB)
        return total_filled == total_boxes

    def best_child(self, exploration_weight=1.41):
        """
        Select best child using UCB1 formula
        UCB1 = (wins / visits) + C * sqrt(ln(parent_visits) / visits)
        """
        choices_weights = []

        for child in self.children:
            if child.visits == 0:
                ucb1_score = float('inf')
            else:
                exploitation = child.wins / child.visits
                exploration = exploration_weight * math.sqrt(math.log(self.visits) / child.visits)
                ucb1_score = exploitation + exploration

            choices_weights.append(ucb1_score)

        return self.children[choices_weights.index(max(choices_weights))]

    def most_visited_child(self):
        """Return the child with the most visits"""
        return max(self.children, key=lambda c: c.visits)

    def expand(self):
        """Add a new child node for an untried move"""
        move = self.untried_moves.pop()
        new_state = self.deep_copy_state()

        # Apply the move to the copied state
        r, c, edge_type = move
        new_state.set_edge(r, c, edge_type)
        new_state.check_box(r, c, edge_type, self.current_player) # Pass current_player

        # Determine the player for the child node
        next_player_for_child = self.current_player if new_state.On_Offensive else (3 - self.current_player)

        child_node = MCTSNode(new_state, next_player_for_child, parent=self, move=move)
        self.children.append(child_node)
        return child_node

    def deep_copy_state(self):
        """Create a deep copy of game state"""
        n = self.game_state.n
        new_game = dots_and_boxes(n)
        new_game.board_tensor = self.game_state.board_tensor.copy()
        new_game.boxes_playerA = self.game_state.boxes_playerA.copy()
        new_game.boxes_playerB = self.game_state.boxes_playerB.copy()
        new_game.recent_edge = self.game_state.recent_edge
        new_game.On_Offensive = self.game_state.On_Offensive
        return new_game

    def simulate(self):
        """
        Simulate a game from this node until the end (ROLLOUT)
        Uses semi-intelligent policy: prioritize box-completing moves
        Returns the result from the perspective of self.current_player (1.0 for win, 0.5 for draw, 0.0 for loss)
        """
        state = self.deep_copy_state()
        sim_current_player = self.current_player

        while True:
            n = state.n
            total_boxes = n ** 2
            total_filled = len(state.boxes_playerA) + len(state.boxes_playerB)

            if total_filled == total_boxes:
                # Game over - return result from the perspective of self.current_player
                player1_final_score = len(state.boxes_playerA)
                player2_final_score = len(state.boxes_playerB)

                if player1_final_score > player2_final_score:
                    winner = 1
                elif player2_final_score > player1_final_score:
                    winner = 2
                else:
                    winner = 0 # Draw

                if self.current_player == winner:
                    return 1.0 # Node's player won the simulated game
                elif winner == 0:
                    return 0.5 # Draw
                else:
                    return 0.0 # Node's player lost the simulated game

            # Get legal moves and categorize them
            legal_moves = []
            completing_moves = []
            safe_moves = []

            for r in range(n):
                for c in range(n):
                    for edge_type in ["top", "bottom", "left", "right"]:
                        if state.check_edge(r, c, edge_type):
                            move = (r, c, edge_type)
                            legal_moves.append(move)

                            if self.move_completes_box(state, move, sim_current_player):
                                completing_moves.append(move)
                            elif not self.move_gives_box(state, move, sim_current_player):
                                safe_moves.append(move)

            if not legal_moves:
                return 0.5

            # Smart move selection
            if completing_moves and random.random() < 0.8:
                move = random.choice(completing_moves)
            elif safe_moves and random.random() < 0.6:
                move = random.choice(safe_moves)
            else:
                move = random.choice(legal_moves)

            r, c, edge_type = move
            state.set_edge(r, c, edge_type)
            _ = state.check_box(r, c, edge_type, sim_current_player)

            # Switch player if no boxes were completed
            if not state.On_Offensive:
                sim_current_player = 3 - sim_current_player

    def move_completes_box(self, state, move, player):
        """Check if a move would complete at least one box for the given player"""
        r, c, edge_type = move
        temp_state = self.deep_copy_state_from(state)
        temp_state.set_edge(r, c, edge_type)

        n = temp_state.n
        completed = 0

        if np.all(temp_state.board_tensor[r, c, :4] == 1) and temp_state.board_tensor[r, c, 4] == 0:
            completed += 1

        if edge_type == "top" and r > 0:
            if np.all(temp_state.board_tensor[r-1, c, :4] == 1) and temp_state.board_tensor[r-1, c, 4] == 0:
                completed += 1
        elif edge_type == "bottom" and r < n-1:
            if np.all(temp_state.board_tensor[r+1, c, :4] == 1) and temp_state.board_tensor[r+1, c, 4] == 0:
                completed += 1
        elif edge_type == "left" and c > 0:
            if np.all(temp_state.board_tensor[r, c-1, :4] == 1) and temp_state.board_tensor[r, c-1, 4] == 0:
                completed += 1
        elif edge_type == "right" and c < n-1:
            if np.all(temp_state.board_tensor[r, c+1, :4] == 1) and temp_state.board_tensor[r, c+1, 4] == 0:
                completed += 1

        return completed > 0

    def move_gives_box(self, state, move, player):
        """Check if a move creates a box with 3 sides (giving opponent an opportunity)"""
        r, c, edge_type = move
        temp_state = self.deep_copy_state_from(state)
        temp_state.set_edge(r, c, edge_type)

        n = temp_state.n

        for box_r in range(n):
            for box_c in range(n):
                if temp_state.board_tensor[box_r, box_c, 4] == 0:
                    sides = np.sum(temp_state.board_tensor[box_r, box_c, :4])
                    if sides == 3:
                        return True
        return False

    def deep_copy_state_from(self, state):
        """Create a deep copy of a given game state"""
        n = state.n
        new_game = dots_and_boxes(n)
        new_game.board_tensor = state.board_tensor.copy()
        new_game.boxes_playerA = state.boxes_playerA.copy()
        new_game.boxes_playerB = state.boxes_playerB.copy()
        new_game.recent_edge = state.recent_edge
        new_game.On_Offensive = state.On_Offensive
        return new_game

    def backpropagate(self, result):
        """Update node statistics up the tree"""
        node = self

        while node is not None:
            node.visits += 1
            node.wins += result
            node = node.parent


class pureDotsAndBoxesMCTS:
    """MCTS-based AI for Dots and Boxes"""

    def __init__(self, time_limit=1.0, simulation_limit=None):
        """
        Args:
            time_limit: Maximum time (seconds) to search
            simulation_limit: Maximum number of simulations
        """
        self.time_limit = time_limit
        self.simulation_limit = simulation_limit

    def get_move(self, game_state, current_player):
        """
        Get the best move using MCTS

        Args:
            game_state: The current state of the game.
            current_player: The player whose turn it is (1 or 2).

        Returns: tuple (r, c, edge_type)
        """
        root = MCTSNode(game_state, current_player)

        completing_moves = []
        for move in root.untried_moves:
            if root.move_completes_box(root.game_state, move, current_player):
                completing_moves.append(move)

        if completing_moves:
            print(f"Found {len(completing_moves)} box-completing moves! Taking one immediately.")
            return completing_moves[0]

        if len(root.untried_moves) == 1:
            return root.untried_moves[0]

        start_time = time.time()
        simulations = 0

        while True:
            if self.simulation_limit and simulations >= self.simulation_limit:
                break
            if time.time() - start_time >= self.time_limit:
                break

            node = root

            # PHASE 1: SELECTION
            while not node.is_terminal() and node.is_fully_expanded():
                node = node.best_child()

            # PHASE 2: EXPANSION
            if not node.is_terminal() and not node.is_fully_expanded():
                node = node.expand()

            # PHASE 3: SIMULATION
            result = node.simulate()

            # PHASE 4: BACKPROPAGATION
            node.backpropagate(result)

            simulations += 1

        if not root.children:
            return random.choice(root.untried_moves) if root.untried_moves else None

        best_child = None
        best_score = -float('inf')

        for child in root.children:
            if child.visits == 0:
                continue

            win_rate = child.wins / child.visits
            score = win_rate

            if score > best_score:
                best_score = score
                best_child = child

        if best_child is None:
            best_child = root.most_visited_child()

        print(f"MCTS completed {simulations} simulations in {time.time() - start_time:.2f}s")
        if best_child:
            print(f"Best move: {best_child.move} (visits: {best_child.visits}, win rate: {best_child.wins/best_child.visits:.2%})")
        else:
            print("No best child found, falling back to random move or None.")

        return best_child.move if best_child else None


---
# API Setup - End of Document

In [ ]:
import secrets
import json
import os

def generate_api_key():
    return secrets.token_urlsafe(32)

def update_index_html_api_key(new_api_key, html_path="templates/index.html"):
    import re

    with open(html_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Regex to find the current API_KEY assignment and replace the value
    pattern = r"(const\s+API_KEY\s*=\s*)(['\`])([^'\`;]*)(['\`;]+)"
    new_content = re.sub(pattern, rf"\1\2{new_api_key}\2;", content)

    with open(html_path, "w", encoding="utf-8") as f:
        f.write(new_content)

def setup_api(notebook_filename='Dots_and_Boxes_Model.ipynb'): # change name to file name
    api_key = generate_api_key()

    config = {
        "api_key": api_key,
        "port": 5000,
        "host": "0.0.0.0",
        "notebook_path": notebook_filename
    }

    config_path = 'api_config.json' # saves config
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)

    html_path = os.path.join("templates", "index.html")
    update_index_html_api_key(api_key, html_path)

    return api_key

In [10]:
api_key = setup_api(notebook_filename='Dots_and_Boxes_Model.ipynb')
print(f"Your API Key: {api_key}")
print("\nDownload these files:\n1.This Notebook\n2.notebook_to_api.py\n3.requirements.txt\n4.templates/index.html")
print("\nOpen command prompt & run:\npip install -r requirements.txt\npython notebook_to_api.py")
print("\nOpen browser on http://localhost:5000")

Your API Key: ySt_tkkh5fwOmcGpWbl0VLrl7gAJkBiyKNVR-Zj-GNk

Download these files:
1.This Notebook
2.notebook_to_api.py
3.requirements.txt
4.templates/index.html

Open command prompt & run:
pip install -r requirements.txt
python notebook_to_api.py

Open browser on http://localhost:5000
